# Low pressure stimulation of subsurface reservoir
Description: This notebook presents a weakly coupled flow, mechanics and fracture deformation problem reported in the paper *PorePy: An Open-Source Simulation Tool for Flow and Transport in Deformable Fractured Rocks*, by
Eirik Keilegavlen, Alessio Fumagalli, Runar Berge, Ivar Stefansson, Inga Berre. See arXiv:1712:00460 for details. The code in the notebook was that used to produce figure 7 in the paper, and if ran on a separate system, (almost) the same results should result - factors such as grid generation, linear solvers etc. may produce minor differences.

To navigate quickly through the procedure, the main steps are: i) Create a mesh, ii) assign parameters for both flow and transport problems, iii) set up appropriate solvers, iv) discretize and solve. 

The equations we solve are:
Flow:
$$ \phi c_f \frac{\partial p}{\partial t} - \nabla \cdot \mathcal K\nabla p  = q$$

Elasticity:
$$ \nabla \cdot \sigma = 0$$

Friction:
$$ T_s \le \mu (T_n - p)$$

## Preliminaries
To run this, you need to have PorePy installed and set up with Gmsh. The simulations will be time consuming. 

## Imports
The first step is to import all dependencies

In [1]:
# External modules
import numpy as np
import scipy.sparse as sps

# Porepy modules
from porepy.fracs import meshing, fractures
from porepy.params.units import *
from porepy.params import tensor, bc, rock
from porepy.grids.grid import FaceTag
from porepy.numerics.compressible import SlightlyCompressibleModel, SlightlyCompressibleDataAssigner
from porepy.numerics.mechanics import StaticModel, StaticDataAssigner
from porepy.numerics.fracture_deformation import FrictionSlipModel, FrictionSlipDataAssigner
from porepy.utils.comp_geom import compute_normal

ModuleNotFoundError: No module named 'porepy'

## Grid generation
The below function creates a 3D fracture network from a a set of elliptic fractures given in the data file fractures.csv.

In [2]:
def create_grid():
    file_name = 'fractures.csv'  
    data = np.genfromtxt(file_name, delimiter=',')
    data = np.atleast_2d(data)
    centers = data[:, 0:3]
    maj_ax = data[:, 3]
    min_ax = data[:, 4]
    maj_ax_ang = data[:, 5]
    strike_ang = data[:, 6]
    dip_ang = data[:, 7]
    if data.shape[1] == 9:
        num_points = data[:, 8]
    else:
        num_points = 16 * np.ones(data.shape[0])

    frac_list = []

    for i in range(maj_ax.shape[0]):
        frac_list.append(fractures.EllipticFracture(centers[i, :],
                                                    maj_ax[i],
                                                    min_ax[i],
                                                    maj_ax_ang[i],
                                                    strike_ang[i],
                                                    dip_ang[i],
                                                    num_points[i]))
    frac_network = fractures.FractureNetwork(frac_list)
    box = {'xmin': -5000, 'ymin': -5000, 'zmin': -5000,
           'xmax': 10000, 'ymax':  10000, 'zmax': 10000}
    gb = meshing.simplex_grid(frac_network, box, h_ideal=10000, h_min = 200)
    return gb


## Problem setup: Parameter specification and solvers
To set up the flow model, we will use a SlightlyCompressibleModel. The model automatically sets up simple discretization scheme, and tries to design decent linear solvers for the resulting systems of equations.

### Pressure data
The first step is to provide simulation data. Simulation parameters are stored as part of the GridBucket, but accessing this can be somewhat cumbersome. To assist the assignment, and also to provide a reasonable way of setting default parameters, each of the models (pre-defined solvers), are accompanied by a DataAssigner. This can be used directly to define a simulaiton with default parameters, or modified as desired. Below, we give an example for the pressure equation.

In [4]:
# The units module contains various physical constants
# Set a relative high matrix permeability, this would correspond to
# a quite high density of upscaled fractures.
class MatrixDomain(SlightlyCompressibleDataAssigner):
    """ Set data for the 3D domain (matrix) in the pressure equation.
    
    Fields that are not assigned here, will have the default values 
    prescribed in SlighlyCompressibleDataAssigner (which again may point further to defaults
    in the Parameter class).
    """
    def initial_condition(self):
        p = 40 * MEGA * PASCAL
        return p * np.ones(self.grid().num_cells)

    def compressibility(self):
        return 4.6e-10 / PASCAL

    def permeability(self):
        kxx = np.ones(self.grid().num_cells) * NANO * DARCY
        return tensor.SecondOrder(3, kxx / self.viscosity())

    def viscosity(self):
        return .45 * MILLI * PASCAL * SECOND

    def porosity(self):
        return 0.01 * np.ones(self.grid().num_cells)

    def density(self):
        return 1014 * KILOGRAM / METER**3


class FractureDomain(MatrixDomain):
     """ Set data for the fractures (2D 1D and 0D domain) in the pressure equation.
    
    We let this class inherit it's parameters from the Matrix parameters, but we overload
    the appropriate functions
    """
    def __init__(self, g, data):
        self.E0 = .1 * MILLI * METER * np.ones(g.num_cells)
        MatrixDomain.__init__(self, g, data)

    def aperture(self):
        """
        Note that the power is set according to the dimension, thus an 
        intersection will have an initial transverse area of .1 mm squared.
        """
        return self.E0**(3 - self.grid().dim)

    def permeability(self):
        """
        We set the permeability according to the qubic law. The permeability in the 
        intersections will have the same permeability as the fractures
        """
        kxx = self.aperture()**2 / 12
        return tensor.SecondOrder(3, kxx / self.viscosity())

    def porosity(self):
        return 1 * np.ones(self.grid().num_cells)


class InjectionDomain(FractureDomain):
    """
    We will inject in one fracture. This fracture inherits all it's parameters from the 
    fracture class, and we define a source term in the middle of the fracture.
    """
    def source(self, t):
        tol = 1e-4
        value = np.zeros(self.grid().num_cells)

        cell_coord = np.atleast_2d(np.array([1200, 2200, 2000])).T
        distance = np.sqrt(np.sum(np.abs(self.grid().cell_centers - cell_coord)**2, axis=0))
        cell = np.argmin(distance)

        if t < 4200 * SECOND + 1e-6:
            value[cell] = 10.0 * KILOGRAM / SECOND / self.density()
        return value

### Mechanics data
Next, we define the simulation data for the linear elasticity problem: $\nabla\cdot  \sigma = 0$

In [3]:
class MechDomain(StaticDataAssigner):
    """ Set data for the 3D domain (matrix) for the linear elasticity.
    
    Fields that are not assigned here, will have the default values 
    prescribed in StaticDataAssigner (which again may point further to defaults
    in the Parameter class).
    """   
    def bc(self):
        """
        The default boundary condition is Neuman, so we overload this function 
        to define zero Dirichlet condition on the boundary. 
        """
        bc_cond = bc.BoundaryCondition(
            self.grid(), self.grid().get_boundary_faces(), 'dir')
        return bc_cond

    def stress_tensor(self):
        """
        We set the stress tensor based on the parameters assigned to the Rock class
        """
        mu = self.data()['rock'].MU * np.ones(self.grid().num_cells)
        lam = self.data()['rock'].LAMBDA * np.ones(self.grid().num_cells)
        return tensor.FourthOrder(self.grid().dim, mu, lam)

    def background_stress(self):
        """
        The background stress defines stress tensor, and we assume the same stress
        throughout our domain
        """
        T_x = .120 * GIGA * PASCAL
        T_y = .080 * GIGA * PASCAL
        T_z = .100 * GIGA * PASCAL
        sigma = -np.array([[T_x, 0, 0], [0, T_y, 0], [0, 0, T_z]])
        return sigma


NameError: name 'StaticDataAssigner' is not defined

### Assign data
Having defined parameter classes for all geometric objects, assigning the data is easy: Simply loop over the GridBucket, and choose DataAssigner according to the grid dimension.

In [6]:
# Define method to assign parameters to all nodes in the GridBucket
def assign_data(gb):
    # First we define the rock
    matrix_rock = rock.Granite()
    matrix_rock.MU = 20 * GIGA * PASCAL
    matrix_rock.LAMBDA = 20 * GIGA * PASCAL
    
    # We define the variable aperture_change which will be used to update the aperture
    # at each time step
    gb.add_node_props(['aperture_change'])
    for g, d in gb:
        d['aperture_change'] = np.zeros(g.num_cells)
        if g.dim == 3:
            d['rock'] = matrix_rock
            d['flow_data'] = MatrixDomain(g, d)
            d['mech_data'] = MechDomain(g, d)
            d['slip_data'] = FrictionSlipDataAssigner(g, d)
        else:
            # We define an injection in the first fracture
            if d['node_number'] == 1:
                d['flow_data'] = InjectionDomain(g, d)
            else:
                d['flow_data'] = FractureDomain(g, d)



# Set up solvers
We are finally ready to define our solver objects and solve for flow and temperature. With all parameters defined, this is a relatively simple code:

In [11]:
# Define computaitonal mesh (mixed-dimensional)
gb = create_grid()

# Create an exporter object, and dump the grid
exporter = Exporter(gb, 'grid')
exporter.write_vtk()

Construct mesh
Use existing decomposition
Minimal distance between points encountered is 0.000320273385469
Gmsh processed file successfully


Grid creation completed. Elapsed time 2.2307803630828857


Created 1 3-d grids with 243352 cells
Created 66 2-d grids with 28113 cells
Created 50 1-d grids with 256 cells


Done. Elapsed time 51.18879413604736
Assemble in bucket


/home/eke001/Dropbox/workspace/python/porepy/src/porepy/fracs/utils.py:247: UserWarning: Found inconsistency between cells and higher
                          dimensional faces. Continuing, fingers crossed
  dimensional faces. Continuing, fingers crossed''')


Done. Elapsed time 11.762894630432129
Compute geometry
Done. Elapsed time 1.3059687614440918
Split fractures
Done. Elapsed time 369.0356047153473
Mesh construction completed. Total time 433.33227157592773


The resulting grid looks like this, after some manipulation in Paraview

In [12]:
display(HTML("<table><tr><td><img src='fig/g0.png'></td><td><img src='fig/g1.png'></td></tr></table>"))

,


### Define solvers
The flow problem is dependent on time, and needs the time step as an argument. The mechanics and fracture deformation are both quasi-static, i.e., slip happens instantaneous when the Mohr-Colomb criterion is violated

In [13]:
# Define the time stepping
dt = 7 * MINUTE
T = 18 * dt

# Define pressure solver for the given grid.
# This will assign parameters, using the above classes
flow_solver = SlightlyCompressibleModel(gb, time_step=dt)
mech_solver = StaticModel(g3, data3)
friction_solver = FrictionSlipModel(g3, data3)

2017-12-14 09:39:20,134 - porepy.numerics.elliptic - Solve elliptic model
2017-12-14 09:39:20,135 - porepy.numerics.elliptic - Discretize
2017-12-14 09:39:46,767 - porepy.numerics.elliptic - Done. Elapsed time 26.63194513320923
2017-12-14 09:39:46,768 - porepy.numerics.elliptic - Solve linear system using GMRES
2017-12-14 10:09:30,355 - porepy.numerics.elliptic - GMRES succeeded.
2017-12-14 10:09:30,358 - porepy.numerics.elliptic - Done. Elapsed time 1783.5909671783447


### Transfer data
The linear elasticity and fracture deformation models are defined on the 3D grid. The traction calculations and shear and normal deformation will take place on the faces of the 3D grid that are connected to the 2D fracture cells. For the flow problem, however, the aperture and pressure are defined in the cells of the 2D fractures. We therefore need two simple functions that map data from the cells of 2D grids to faces of 3D grids and vice versa

In [6]:
def cell_2_face(gb, variable):
    g3 = gb.grids_of_dimension(3)[0]
    data3 = gb.node_props(g3)
    face_variable = np.zeros(g3.num_faces)
    for g, d in gb:
        if g.dim != 2:
            continue
        f_c = gb.edge_prop([(g3, g)], 'face_cells')[0]
        ci, fi, _ = sps.find(f_c)
        face_variable[fi] = d[variable][ci]

    data3['face_' + variable] = face_variable

def face_2_cell(gb, variable):
    g3 = gb.grids_of_dimension(3)[0]
    data3 = gb.node_props(g3)
    for g, d in gb:
        if g.dim != 2:
            continue
        f_c = gb.edge_prop([(g3, g)], 'face_cells')[0]
        ci, fi, _ = sps.find(f_c)

        cell_variable = np.zeros(g.num_cells)
        num_hit = np.zeros(g.num_cells)
        for i, face in enumerate(fi):
            cell_variable[ci[i]] += data3[variable][face]
            num_hit[ci[i]] += 1
        d[variable] = cell_variable / num_hit


### Aperture update

At each time step the fracture may possible slip. If a fracture do slip we will get an equivalent increse in aperture. For convenience, we define a function that updates the aperture based on this aperture increase

In [7]:
def update_aperture(gb, name='aperture_change'):
    for g, d in gb:
        if g.dim != 2:
            continue
        E0 = d['flow_data'].E0
        d['param'].set_aperture(E0 + d[name])

## Define Time loop

In [ ]:
# Export the initial pressure
flow_solver.pressure('pressure')
exporter.write_vtk(['pressure'], time_step=0.)

time_steps = []
time_steps.append(t)
k = 0

while t < T:
    t += dt
    k += 1
    time_steps.append(t)

    # Solve flow
    flow_solver._solver.update(t)    # Update injection
    flow_solver.reassemble()         # Reasemble rhs
    flow_solver.step()               # solve for next time step
    flow_solver.pressure('pressure') # save solution to data
    cell_2_face(gb, 'pressure')      # map cell pressure to 3D faces

    # solve mechanics
    do_slip = True
    # At the start of each time step we assume no fractures are slipping
    friction_solver.is_slipping = np.zeros(g3.num_faces, dtype=np.bool)
    it_num = 0
    while np.any(do_slip):
        it_num += 1
        mech_solver.solve(discretize=False)
        mech_solver.traction('traction')
        do_slip = friction_solver.step()
        data3['param'].set_slip_distance(friction_solver.x.ravel('F'))
        if do_slip:
            print('Fracture is slipping. Iteration #: ', it_num)
 
    friction_solver.aperture_change('aperture_change')  # Save aperture change to data
    face_2_cell(gb, 'aperture_change')                  # Map aperture change to 2D cells
    update_aperture(gb)                                 # Update the aperture
    exporter.write_vtk(['pressure', 'aperture_change'], time_step=k)

exporter.write_pvd(np.array(time_steps))


Here is what the pressure solution looks like

In [14]:
display(HTML("<table><tr><td><img src='fig/pressure.png'></td><td><img src='fig/p2.png'></td></tr></table>"))

,


And we are done. Visualized, the transport solution evolves like this

In [16]:
HTML('<img src="fig/out.gif">')